In [1]:
import os
import sys
import datetime
import pytz
import json

import numpy as np
from scipy.signal import medfilt
import matplotlib.pyplot as plt
import polars as pl
import pandas as pd

import synoptic

from brc_tools.utils.lookups import _VRBLS

# JRL: if problems, delete your existing config toml file
# then create new one by uncommenting:
# synoptic.configure(token="blah")

# Use Helvetica or Arial for plots
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams['font.size'] = 12


# Importing observation data from Synoptic Weather as polars dataframe.
We then save as json to send to our team's UBAIR website server.

The plan goes as follows:
- Import data from Synoptic Weather based on date, location, variable, etc
- Export a json file from the polars (?) dataframe
- Note the formatting so we can make the website read it in a predictable format

First off: we want time series for multiple stations.

Some of these data are noisy and/or with errors. We will filter some variables later.

In [2]:
stid_list = ["KSLC", "UTORM", "CLN", "UTHEB", "UTCOP", "UTSTV", "UBHSP", "UB7ST", "UBCSP",
             # 'COOPDINU1', 'COOPROSU1',  'COOPVELU1',
             'COOPFTDU1', 'COOPALMU1', 'COOPDSNU1', 'COOPNELU1',
             ]
data_root = "./data"
data_fname = "df_obs_pp.h5"
metadata_fname = "df_metadata.h5"
df_obs_fpath = os.path.join(data_root, data_fname)
df_meta_fpath = os.path.join(data_root, metadata_fname)

start_date = datetime.datetime(2025, 1, 24, 0, 0, 0)
end_date = datetime.datetime(2025, 2, 4, 0, 0, 0)
# end_date = datetime.datetime(2025, 3, 16, 0, 0, 0)

# df_meta = load_pickle(df_meta_fpath)
# df_obs = pd.read_hdf(df_obs_fpath, key='df_obs')
# df_obs_winter = df_obs[df_obs.index.month.isin([11, 12, 1, 2, 3])]


In [3]:
def replace_max_values(df, vrbl, max_value=None):
    # Assumes df is already filtered by stid.
    if max_value is None:
        max_value = df.select(pl.col(vrbl).max())[0, 0]
    df = df.with_columns(
        pl.when(pl.col(vrbl) == max_value)
        .then(None)
        .otherwise(pl.col(vrbl))
        .alias(vrbl)
    )
    return df.with_columns(pl.col(vrbl).interpolate())

def apply_median_filter(df, vrbl, kernel_size):
    # Convert column to numpy array and apply median filter.
    # Filtered by stid already
    arr = df[vrbl].to_numpy().astype("float32")
    med_filtered = medfilt(arr, kernel_size=kernel_size)
    return df.with_columns(pl.Series(name=vrbl, values=med_filtered))

def filter_snow_depth(df, kernel_size):
    # Run the preprocessing steps in sequence.
    df = replace_max_values(df, "snow_depth")
    df = apply_median_filter(df, "snow_depth", kernel_size=kernel_size)
    return df

def plot_snow_depth(ax, df, stid, kernel_size=5):
    # Filter rows using Polars' filter method.
    df_filtered = df.filter(pl.col("stid") == stid)

    if kernel_size is not None:
        df_filtered = filter_snow_depth(df_filtered, kernel_size=kernel_size)
        df_filtered = df_filtered.with_columns(pl.col("snow_depth").interpolate())

    # Convert time to Mountain Time Zone.
    df_filtered = df_filtered.with_columns(pl.col("date_time").dt.convert_time_zone("America/Denver"))
    # Make linestyle dashed if stid begins with "COOP"; else use solid
    ls = "--" if stid.startswith("COOP") else "-"
    ax.plot(df_filtered["date_time"], df_filtered["snow_depth"], label=f"{stid}", alpha=0.5, lw=0.75,
            linestyle=ls)



In [4]:
# Collect all strings for the "synoptic" key from the nested dictionaries
synoptic_vrbls = [
    value['synoptic'] for value in _VRBLS.values()]

In [5]:
df_meta = synoptic.Metadata(stid=stid_list, verbose=True).df()
df_meta

🚚💨 Speedy delivery from Synoptic's metadata service.
📦 Received data from 13 stations (0.19 seconds).


id,stid,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,period_of_record_start,period_of_record_end,is_restricted,restricted_metadata,is_active
u32,str,str,f64,f64,f64,u32,str,str,f64,"datetime[μs, UTC]","datetime[μs, UTC]",bool,bool,bool
53,"""KSLC""","""Salt Lake City, Salt Lake City…",4226.0,40.77069,-111.96503,1,"""UT""","""America/Denver""",4235.6,1997-01-01 00:00:00 UTC,2025-07-12 01:20:00 UTC,false,false,true
525,"""CLN""","""ALTA - COLLINS""",9662.0,40.5763,-111.6383,6,"""UT""","""America/Denver""",9629.3,1997-01-01 00:00:00 UTC,2025-07-12 01:00:00 UTC,false,false,true
15542,"""UTSTV""","""US-40 @ Starvation""",5720.0,40.17258,-110.493,4,"""UT""","""America/Denver""",5715.2,2006-03-14 00:00:00 UTC,2025-07-12 01:20:00 UTC,false,false,true
25080,"""UTHEB""","""US-40 Heber""",5733.0,40.5636,-111.4288,4,"""UT""","""America/Denver""",5754.6,2008-12-09 00:00:00 UTC,2025-07-12 01:20:00 UTC,false,false,true
37071,"""UTORM""","""I-15 @ Orem""",4650.0,40.31925,-111.7267,4,"""UT""","""America/Denver""",4652.2,2013-03-13 00:00:00 UTC,2025-07-12 01:20:00 UTC,false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
58707,"""COOPALMU1""","""ALTAMONT""",6456.0,40.36696,-110.29858,74,"""UT""","""America/Denver""",6407.5,2016-07-12 15:13:00 UTC,2025-07-11 01:00:00 UTC,false,false,true
58734,"""COOPDSNU1""","""DUCHESNE""",5551.0,40.1703,-110.3978,75,"""UT""","""America/Denver""",5534.8,2016-06-18 00:53:00 UTC,2025-07-01 00:00:00 UTC,false,false,true
58747,"""COOPFTDU1""","""FORT DUCHESNE""",5052.0,40.2841,-109.8611,75,"""UT""","""America/Denver""",5003.3,2016-06-13 14:18:00 UTC,2025-05-23 14:20:00 UTC,false,false,false


In [6]:
df_data = synoptic.TimeSeries(stid=stid_list,start=start_date, end=end_date,
                                 vars=synoptic_vrbls, verbose=True,
                                 # rename_set_1=False, rename_value_1=False
                                ).df().synoptic.pivot()
df_data.head(20)

🚚💨 Speedy delivery from Synoptic's timeseries service.
📦 Received data from 13 stations (1.42 seconds).


date_time,stid,latitude,longitude,elevation,air_temp,wind_speed,wind_direction,sea_level_pressure,snow_depth,ozone_concentration
"datetime[μs, UTC]",str,f64,f64,f64,f64,f64,f64,f64,f64,f64
2025-01-24 00:00:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,2.0,2.572,160.0,103539.68,null,null
2025-01-24 00:05:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,2.0,2.572,140.0,103539.68,null,null
2025-01-24 00:10:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,2.0,2.572,110.0,103539.68,null,null
2025-01-24 00:15:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,2.0,1.543,140.0,103539.68,null,null
2025-01-24 00:20:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,1.0,2.058,140.0,103598.3,null,null
…,…,…,…,…,…,…,…,…,…,…
2025-01-24 01:10:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,0.6,1.543,130.0,103587.9,null,null
2025-01-24 01:15:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,0.6,2.058,130.0,103587.89,null,null
2025-01-24 01:20:00 UTC,"""KSLC""",40.77069,-111.96503,4226.0,1.1,2.058,150.0,103558.44,null,null


In [7]:
latest_obs = synoptic.Latest(stid=stid_list, vars=synoptic_vrbls, verbose=True,
                within=datetime.timedelta(hours=25), # 1 hour
                             ).df()

# Check for rows with identical stid and variable
# Keep only the most recent.
latest_obs = latest_obs.sort(["stid", "variable", "date_time"], descending=[False, False, True])
latest_obs = latest_obs.unique(subset=["stid", "variable"], keep="first")

latest_obs

🚚💨 Speedy delivery from Synoptic's latest service.
📦 Received data from 9 stations (0.20 seconds).


stid,variable,sensor_index,is_derived,value,date_time,units,id,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,period_of_record_start,period_of_record_end,qc_flagged,is_restricted,restricted_metadata,is_active
str,str,u32,bool,f64,"datetime[μs, UTC]",str,u32,str,f64,f64,f64,u32,str,str,f64,"datetime[μs, UTC]","datetime[μs, UTC]",bool,bool,bool,bool
"""UTORM""","""air_temp""",1,false,31.111,2025-07-12 01:50:00 UTC,"""Celsius""",37071,"""I-15 @ Orem""",4650.0,40.31925,-111.7267,4,"""UT""","""America/Denver""",4652.2,2013-03-13 00:00:00 UTC,2025-07-12 01:20:00 UTC,false,false,false,true
"""UBCSP""","""snow_depth""",1,false,0.0,2025-07-12 01:45:00 UTC,"""Millimeters""",48222,"""Castle Peak""",5266.0,40.051,-110.02,209,"""UT""","""America/Denver""",5262.5,2016-01-29 19:47:00 UTC,2025-07-12 01:15:00 UTC,false,false,false,true
"""UTORM""","""wind_direction""",1,false,327.3,2025-07-12 01:50:00 UTC,"""Degrees""",37071,"""I-15 @ Orem""",4650.0,40.31925,-111.7267,4,"""UT""","""America/Denver""",4652.2,2013-03-13 00:00:00 UTC,2025-07-12 01:20:00 UTC,false,false,false,true
"""UBCSP""","""wind_speed""",1,false,9.96,2025-07-12 01:45:00 UTC,"""m/s""",48222,"""Castle Peak""",5266.0,40.051,-110.02,209,"""UT""","""America/Denver""",5262.5,2016-01-29 19:47:00 UTC,2025-07-12 01:15:00 UTC,false,false,false,true
"""COOPALMU1""","""air_temp""",1,false,26.111,2025-07-12 01:00:00 UTC,"""Celsius""",58707,"""ALTAMONT""",6456.0,40.36696,-110.29858,74,"""UT""","""America/Denver""",6407.5,2016-07-12 15:13:00 UTC,2025-07-11 01:00:00 UTC,false,false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""UBHSP""","""wind_direction""",1,false,7.55,2025-07-12 01:45:00 UTC,"""Degrees""",48227,"""Horsepool""",5148.0,40.144,-109.467,209,"""UT""","""America/Denver""",5157.5,2016-01-29 19:47:00 UTC,2025-07-12 01:15:00 UTC,false,false,false,true
"""COOPALMU1""","""snow_depth""",1,false,0.0,2025-07-12 01:00:00 UTC,"""Millimeters""",58707,"""ALTAMONT""",6456.0,40.36696,-110.29858,74,"""UT""","""America/Denver""",6407.5,2016-07-12 15:13:00 UTC,2025-07-11 01:00:00 UTC,false,false,false,true
"""CLN""","""snow_depth""",1,false,0.0,2025-07-12 01:00:00 UTC,"""Millimeters""",525,"""ALTA - COLLINS""",9662.0,40.5763,-111.6383,6,"""UT""","""America/Denver""",9629.3,1997-01-01 00:00:00 UTC,2025-07-12 01:00:00 UTC,false,false,false,true


In [8]:
# we only want stid, variable, value, date_time, units
latest_obs = latest_obs.select(
    pl.col("stid"),
    pl.col("variable"),
    pl.col("value"),
    pl.col("date_time"),
    pl.col("units")
)

latest_obs = latest_obs.with_columns(
    pl.col("date_time").dt.convert_time_zone("UTC")
).sort(["stid", "date_time"], descending=[False, True])

latest_obs

stid,variable,value,date_time,units
str,str,f64,"datetime[μs, UTC]",str
"""CLN""","""snow_depth""",0.0,2025-07-12 01:00:00 UTC,"""Millimeters"""
"""CLN""","""air_temp""",17.222,2025-07-12 01:00:00 UTC,"""Celsius"""
"""COOPALMU1""","""air_temp""",26.111,2025-07-12 01:00:00 UTC,"""Celsius"""
"""COOPALMU1""","""snow_depth""",0.0,2025-07-12 01:00:00 UTC,"""Millimeters"""
"""KSLC""","""wind_direction""",350.0,2025-07-12 01:54:00 UTC,"""Degrees"""
…,…,…,…,…
"""UTORM""","""wind_direction""",327.3,2025-07-12 01:50:00 UTC,"""Degrees"""
"""UTORM""","""snow_depth""",17.78,2025-07-12 01:50:00 UTC,"""Millimeters"""
"""UTSTV""","""wind_speed""",6.94,2025-07-12 01:50:00 UTC,"""m/s"""


In [9]:
def clean_dataframe_for_json(df):
    # If the dataframe is a Polars dataframe, convert it to Pandas.
    if hasattr(df, "to_pandas"):
        df = df.to_pandas()

    # Replace NaN with None to become proper JSON null.
    df = df.where(pd.notnull(df), None)

    # Clean string columns (remove unnecessary quotes).
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].str.strip('"')

    return df

def export_data(df, filename, orient='records'):
    df = clean_dataframe_for_json(df)

    # Export to JSON.
    with open(filename, 'w') as f:
        json.dump(df.to_dict(orient=orient), f, default=str)

    print(f"Exported {len(df)} records to {filename}")
    return

In [10]:
export_data(latest_obs, "../data/df_obs_test.json")

# I might create one for a subsample (random) or subset by station, etc
# Operationally on UBAIR site, we want  obs for map stations in last hour

FileNotFoundError: [Errno 2] No such file or directory: '../data/df_obs_test.json'

## Visuals

In [ ]:
# Plot snow depth for each station over time
# Each station has a different reporting frequency and/or time, so plot independently
# All stations are in Mountain Time Zone



fig, ax = plt.subplots(figsize=(12, 6))
for stid in stid_list:
    # Skips here
    # if stid in ("KSLC",):

    # Plus for plot zooming
    if stid in ("KSLC","UTCOP","CLN"):
        continue

    if stid.startswith("COOP"):
        ks = None
        # But the snow 24h variable has 0.51 cm while depth has zero! It was cold!
    else:
        ks = 51
    plot_snow_depth(ax, df_data, stid, kernel_size=ks)

ax.set_xlabel("Time")
ax.set_ylabel(_VRBLS["snow"]["label"])
ax.set_title("Case study 2024/2025: high ozone in UB")

# Light grey background
ax.set_facecolor("#f0f0f0")

ax.legend()
ax.grid(False)
plt.show()